# Desarrolle un experimento (mediante Telnet o analizador de paquetes) que muestre si el servidor de HTTP agrega o quita información a la que genera un programa CGI. Nota: debería programar o utilizar un programa CGI para conocer exactamente lo que genera y debería mostrar la información que llega del lado del cliente, a nivel de HTTP o HTML (preferentemente HTTP).

## Experimento con modulo "request" de python

In [1]:
import requests

#r = requests.post("http://localhost/cgi-bin/hola.py")
r = requests.post("http://localhost:8080/cgi-bin/hola.py")
r.encoding
r.status_code


200

In [2]:
 
print(r.headers)


{'Date': 'Mon, 05 Nov 2018 16:28:11 GMT', 'Server': 'Apache/2.4.37 (Unix)', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html'}


## Experimento desde Wireshark
![error](images/pto1.png)

## Conclusion 

Analizando el header, podemos ver que a nivel HTTP se agregan los campos de estado, fecha, servidor y última modificación.


# Se debe desarrollar una aplicación que presente al usuario una página web con una opción de carga de datos y una opción de modificación de datos. La opción de carga consistirá en mostrar un formulario con un conjunto de datos a completar y que luego almacene estos datos en un archivo de texto plano. Los datos son:

+ Nombre y Apellido (hasta 70 caracteres)

+ Número de Alumno/Legajo (cantidad de dígitos limitada)

+ Sexo (lista desplegable para elegir entre Masculino o Femenino)

+ Edad (hasta dos dígitos)

+ Contraseña (que no se muestre en pantalla)


Para el caso de “modificación”, se deberá requerir Numero de Alumno y la Contraseña, si este par es encontrado en el archivo de texto plano, se muestra lo que ya se tiene para ser modificado como si se estuvieran cargando los datos, pero con los valores iniciales/default cargados del archivo. Utilizar cookies para permitir sucesivas modificaciones sobre los datos sin volver a pedir el legajo y palabra clave.

Se deben desarrollar las páginas estáticas HTML, el o los programas CGI para procesar los datos y la distribución de estos componentes en el sistema de archivos.

Justifique si este ejercicio puede ser considerado una aplicación web.

# Agregue a la aplicación del ejercicio 3 cuatro posibilidades de consultas básicas: por nombre y apellido (pueden haber “*”), Número de Alumno/Legajo (puede ser un intervalo), sexo, y edad (puede ser un intervalo). Los resultados deben mostrarse visualmente diferenciados. ej:

| Nombre | Nro  |Edad | Sexo  |
|--------|------|-----|-------|
| nomrbe1|nro1  |edad1|sexo1  |
| nomrbe2|nro2  |edad2|sexo2  |


**COMENTARIO:** este ejercicio es para que se vea que el programa CGI debería combinar la generación de HMTL con el resultado de lo que se procesa localmente en el servidor. Y que el HTML generado puede ser "largo". El programa CGI puede estar en el lenguaje que se desee.

# Agregue a la aplicación la consulta de valores totales por rango de edad (0-20, 20-40, mas de 40) y sexo.

**COMENTARIO:** este ejercicio es para que se vea que la mayor parte del procesamiento en el lado
del servidor puede ser solamente para computar valores simples a mostrar, pero que pueden ser
resultado de procesamiento relativamente complejo.

# Tome el código que se adjunta como Anexo 5 del apunte de AJAX y agregue las siguientes mejora de funcionalidad:

+ **a)** Agregar un botón a la ventana principal de chat, a efectos que un usuario ingresante pueda registrarse en el sitio. El usuario ingresará su “Nick” y al pulsar el botón, se registrará en un archivo data/usuarios.txt en el servidor.

+ **b)** Producido el registro, se devolverá al usuario una lista total de usuarios registrados en una pequeña ventana a la derecha de la ventana común de mensajes y también se devolverá toda la conversación entre usuarios registrada hasta ese momento.

+ **c)** Intentar establecer la actualización incremental del chat, mediante el almacenamiento en cada cliente de la última línea de chat enviada.

# DFS:

+ **a)** Configurar NFS y efectuar un compartimiento de archivos entre un Servidor y un Cliente Linux. Describir la operatoria en el informe.

+ **b)** Configurar Samba y efectuar un compartimiento de archivos entre un Windows y un Cliente Linux. Describir la operatoria en el informe.


## NFS

Un NFS nos permite acceso local a archivos remotos y utiliza una arquitectura estándar cliente-servidor para compartir archivos entre maquinas con SO basados en unix. Aún así, no es necesario que las dos máquinas corran el mismo sistema operativo.

En nuestro caso configuramos NFS para una distribucion ArchLinux. 

Como primer paso debemos instalar el paquete **nfs-utils**. 

```
sudo pacman -S nfs-utils
```
### Configuracion

+ Iniciar y abilitar los servicios de NFS

```
systemctl start nfs-server.service
```
```
systemctl enable nfs-server.service
```
Ahora bien, algunos de los archivos más importantes que entran en juego con NFS son (Con sus rutas absolutas en ArchLinux):
+ /etc/exports: Es el archivo primario de configuración de NFS. Todos los archivos y carpetas exportados que se especifican en este archivo son los que se sirven.
+ /etc/fstab: Archivo de FS de linux en el que se especifican los puntos de montaje de las diferentes particiones. Se utiliza para definir un mounpoint para el/los directorio/s de NFS (Y que persistan luego de reiniciar el sistema).
+ /etc/idmapd.conf: Archivo en el que se realiza el mapeo de IDs. Utilizado principalmente para definir el nombre del dominio para que el servicio se configure correctamente.

### Mapeo de ID

+ Para este paso hay que modificar el archivo de configuracion **/etc/idmapd.conf** y establezca el campo **Domain** con el nombre de dominio.

+ Seguidamente, creamos el recurso que vamos a servir. En este caso es una carpeta que ubicaremos en **/srv/nfs4/datos**

```
sudo mkdir -p /srv/nfs4/datos
```

+ Luego creamos la carpeta del sistema de archivos del servidor en donde será montado el recurso a compartir:


```
sudo mkdir /mnt/datos
```
Se deben otorgar permisos de lectura/escritura al directorio **datos** para que los clientes puedan escribir en él.

+ Luego, como siguiente paso es montar el directorio que desea compartir, en este caso **/mnt/datos**, en el directorio compartido de NFS a través de la instrucción de montaje.

```
mount --bind /mnt/datos /srv/nfs4/datos
```
Para hacer que los cambios sean permanentes en cada reinicio del servidor, añada el enlace de montaje a **fstab**

![error](images/fstab.png)

### Exportacion
Como siguiente paso, se añaden los directorios que se compartirán y la dirección IP o nombre de servidor de las máquinas cliente a las que les estará permitido montarlas en **exports**:

![error](images/exports.png)

Si se modifica **/etc/exports** mientras el servidor está funcionando, debe volver a exportarlos para que los cambios surtan efecto.

```
exportfs -rav
```
![error](images/exportfs.png)

Finalmente, para levantar el servidor, debemos arrancar los servicios **rpcbind** y **nfs-client.target**.


```
systemctl start rpcbind nfs-client.target
```

## SAMBA

Como paso inicial debemos instalar el paquete de **samba**.

```
sudo pacman -S samba
```
Samba se configura en el **/etc/samba/smb.conf**.

Debido a que el paquete samba no proporciona este archivo, se debe crear antes de iniciar samba.

Utilizamos un ejemplo documentado **smb.conf.default** desde el repositorio de git de **Samba** para la configuración **/etc/samba/smb.conf**.

Link:  <https://git.samba.org/samba.git/?p=samba.git;a=blob_plain;f=examples/smb.conf.default;hb=HEAD>

+ Siempre que modifique el smb.confarchivo, es recomendable ejecutar el comando **testparm** para verificar si hay errores sintácticos.

```
testparm
```
![error](images/testparm1.png)
![error](images/testparm2.png)

### Gestión de usuarios

Samba requiere una cuenta de usuario de Linux. Puede usar una cuenta de usuario existente o crear una nueva.
Aunque el nombre de usuario se comparte con el sistema Linux, Samba usa una contraseña separada de la de las cuentas de usuario de Linux. Reemplazar **samba_usercon** la cuenta de usuario Samba elegida:

```
smbpasswd -a samba_user
```

Luego, del lado del cliente (Workgroup del equipo): Debemos asegurarnos que los servicios smbd y nmbd estén corriendo (Ya sea con START o ENABLE). Esto inicia la escucha del servidor.

Ahora es necesario crear el recurso compartido. Hacemos una carpeta en **/var/lib/samba** llamada **usershare** (Quedando como: **/var/lib/samba/usershare**).

Ahora vamos a modificar los permisos sobre esa carpeta. Típicamente lo que se hace es crear un grupo de usuarios que se puede llamar **sambashare** y se le asignan permisos de lectura/escritura sobre el recurso compartido. Paso a paso tenemos:

+ Creamos el grupo sambashare con el comando **groupadd -r sambashare**
+ Cambiamos el grupo dueño de la carpeta compartida con el comando “chown root:sambashare /var/lib/samba/usershare”.
+ Cambiamos los permisos a nivel de FS sobre la carpeta compartida con el comando “chmod 770 /var/lib/samba/usershare”
+ En el archivo de configuración de samba (smb.conf) debemos agregar el recurso compartido. 
Un posible ejemplo de esto podría ser:
```
[global]
 usershare path = /var/lib/samba/usershare
 usershare max shares = 100
 usershare allow guests = yes
 usershare owner only = yes
```

Finalmente, agregamos el usuario al grupo **sambashare** con el comando **gpasswd sambashare -a usuario**. Luego de esto, es necesario reiniciar la sesión del SO para que se apliquen los cambios.
```
gpasswd sambashare -a usuario
```


# DNS:

**Objetivo:** Observar las llamadas implícitas a DNS y ponderar tiempos de acceso. Efectuar el siguiente experimento, verificando información con la ayuda de un analizador de protocolo:

+ **a)** Desde el browser haga una conexión a una página WEB de argentina, que haya utilizado hace mucho tiempo o que no haya utilizado (un diario, un blog, etc.). Obtenga la diferencia temporal entre la consulta que se observa en el analizador y su respuesta.

![error](images/servidor_nacional.png)


+ **b)** Haga lo mismo con una página no frecuentada por Ud. que se supone que está situada en un servidor europeo o de Asia. (la página oficial del grupo Nightwish, o algún diario galés, etc.). Compare sus conclusiones con las obtenidas en el caso a).

Se puede ver claramente que que el tiempo de respuesta es mucho mayor a la respuesta a un servidor nacional (como el del anterior caso).

![error](images/server_externo.png)

+ **c)** Vuelva nuevamente a invocar la página seleccionada en a), (que se supone ya reside en su caché). Compare resultados.

Se puede ver la significativa mejora en el tiempo de respuesta de la peticion, esto es porque la pagina reside en nuestra cache. Siempre que una pagina resida en la cache, la respuesta sera mucho mas rapida.

![error](images/cacheado.png)

+ **d)** Aporte las conclusiones que puede sacar del experimento.

Como pudimos observar en las distintas experiencias, es más rápido resolver una consulta de una página nacional (www.lanacion.com.ar) que una europea (www.fortnite.com). Sin embargo los tiempos de respuesta cambian favorablemente cuando se encuentran cacheadas las páginas, las respuestas son significativamente mas rapidas.